# Deadly Visualizations!!!

![Image](../images/viz_types_portada.png)

## Setup

First we need to create a basic setup which includes:

- Importing the libraries.

- Reading the dataset file (source [Instituto Nacional de Estadística](https://www.ine.es/ss/Satellite?L=es_ES&c=Page&cid=1259942408928&p=1259942408928&pagename=ProductosYServicios%2FPYSLayout)).

- Create a couple of columns and tables for the analysis.

__NOTE:__ some functions were already created in order to help you go through the challenge. However, feel free to perform any code you might need.

In [1]:
# imports

import sys
import re
sys.path.insert(0, "../modules")

import numpy as np
import pandas as pd

import plotly.express as px
import cufflinks as cf
cf.go_offline()

import module as mod     # functions are include in module.py

In [2]:
# read dataset

deaths = pd.read_csv('../data/7947.csv', sep=';', thousands='.')

deaths.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 301158 entries, 0 to 301157
Data columns (total 5 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   Causa de muerte  301158 non-null  object
 1   Sexo             301158 non-null  object
 2   Edad             301158 non-null  object
 3   Periodo          301158 non-null  int64 
 4   Total            301158 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 11.5+ MB


In [3]:
# add some columns...you'll need them later

deaths['cause_code'] = deaths['Causa de muerte'].apply(mod.cause_code)
deaths['cause_group'] = deaths['Causa de muerte'].apply(mod.cause_types)
deaths['cause_name'] = deaths['Causa de muerte'].apply(mod.cause_name)

deaths.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 301158 entries, 0 to 301157
Data columns (total 8 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   Causa de muerte  301158 non-null  object
 1   Sexo             301158 non-null  object
 2   Edad             301158 non-null  object
 3   Periodo          301158 non-null  int64 
 4   Total            301158 non-null  int64 
 5   cause_code       301158 non-null  object
 6   cause_group      301158 non-null  object
 7   cause_name       301158 non-null  object
dtypes: int64(2), object(6)
memory usage: 18.4+ MB


In [4]:
deaths.head()

,Causa de muerte,Sexo,Edad,Periodo,Total,cause_code,cause_group,cause_name
0,001-102 I-XXII.Todas las causas,Total,Todas las edades,2018,427721,001-102,Multiple causes,I-XXII.Todas las causas
1,001-102 I-XXII.Todas las causas,Total,Todas las edades,2017,424523,001-102,Multiple causes,I-XXII.Todas las causas
2,001-102 I-XXII.Todas las causas,Total,Todas las edades,2016,410611,001-102,Multiple causes,I-XXII.Todas las causas
3,001-102 I-XXII.Todas las causas,Total,Todas las edades,2015,422568,001-102,Multiple causes,I-XXII.Todas las causas
4,001-102 I-XXII.Todas las causas,Total,Todas las edades,2014,395830,001-102,Multiple causes,I-XXII.Todas las causas


In [5]:
len(deaths['cause_group'].unique())
deaths['cause_code'].unique()

array(['001-102', '001-008', '001', '002', '003', '004', '005', '006',
       '007', '008', '009-041', '009', '010', '011', '012', '013', '014',
       '015', '016', '017', '018', '019', '020', '021', '022', '023',
       '024', '025', '026', '027', '028', '029', '030', '031', '032',
       '033', '034', '035', '036', '037', '038', '039', '040', '041',
       '042-043', '042', '043', '044-045', '044', '045', '046-049', '046',
       '047', '048', '049', '050-052', '050', '051', '052', '053-061',
       '053', '054', '055', '056', '057', '058', '059', '060', '061',
       '062-067', '062', '063', '064', '065', '066', '067', '068-072',
       '068', '069', '070', '071', '072', '073', '074-076', '074', '075',
       '076', '077-080', '077', '078', '079', '080', '081', '082',
       '083-085', '083', '084', '085', '086-089', '086', '087', '088',
       '089', '090-102', '090', '091', '092', '093', '094', '095', '096',
       '097', '098', '099', '100', '101', '102'], dtype=object)

In [6]:
# lets check the categorical variables

var_list = ['Sexo', 'Edad', 'Periodo', 'cause_code', 'cause_name', 'cause_group']

categories = mod.cat_var(deaths, var_list)
categories

,categorical_variable,number_of_possible_values,values
0,cause_code,117,"[001-102, 001-008, 001, 002, 003, 004, 005, 00..."
1,cause_name,117,"[I-XXII.Todas las causas, I.Enfermedades infec..."
2,Periodo,39,"[2018, 2017, 2016, 2015, 2014, 2013, 2012, 201..."
3,Edad,22,"[Todas las edades, Menos de 1 año, De 1 a 4 añ..."
4,Sexo,3,"[Total, Hombres, Mujeres]"
5,cause_group,2,"[Multiple causes, Single cause]"


In [7]:
# we need also to create a causes table for the analysis

causes_table = deaths[['cause_code', 'cause_name']].drop_duplicates().sort_values(by='cause_code').reset_index(drop=True)

causes_table

,cause_code,cause_name
0,001,Enfermedades infecciosas intestinales
1,001-008,I.Enfermedades infecciosas y parasitarias
2,001-102,I-XXII.Todas las causas
3,002,Tuberculosis y sus efectos tardíos
4,003,Enfermedad meningocócica
...,...,...
112,098,Suicidio y lesiones autoinfligidas
113,099,Agresiones (homicidio)
114,100,Eventos de intención no determinada
115,101,Complicaciones de la atención médica y quirúrgica


In [8]:
# And some space for free-style Pandas!!! (e.g.: df['column_name'].unique())

# De qué principales causas moría la gente en 1982
deaths[deaths['Periodo'] == 1982]



,Causa de muerte,Sexo,Edad,Periodo,Total,cause_code,cause_group,cause_name
36,001-102 I-XXII.Todas las causas,Total,Todas las edades,1982,286655,001-102,Multiple causes,I-XXII.Todas las causas
75,001-102 I-XXII.Todas las causas,Total,Menos de 1 año,1982,5822,001-102,Multiple causes,I-XXII.Todas las causas
114,001-102 I-XXII.Todas las causas,Total,De 1 a 4 años,1982,1242,001-102,Multiple causes,I-XXII.Todas las causas
153,001-102 I-XXII.Todas las causas,Total,De 5 a 9 años,1982,926,001-102,Multiple causes,I-XXII.Todas las causas
192,001-102 I-XXII.Todas las causas,Total,De 10 a 14 años,1982,851,001-102,Multiple causes,I-XXII.Todas las causas
...,...,...,...,...,...,...,...,...
300999,102 Otras causas externas y sus efectos tardíos,Mujeres,De 75 a 79 años,1982,0,102,Single cause,Otras causas externas y sus efectos tardíos
301038,102 Otras causas externas y sus efectos tardíos,Mujeres,De 80 a 84 años,1982,1,102,Single cause,Otras causas externas y sus efectos tardíos
301077,102 Otras causas externas y sus efectos tardíos,Mujeres,De 85 a 89 años,1982,0,102,Single cause,Otras causas externas y sus efectos tardíos
301116,102 Otras causas externas y sus efectos tardíos,Mujeres,De 90 a 94 años,1982,1,102,Single cause,Otras causas externas y sus efectos tardíos


## Lets make some transformations

Eventhough the dataset is pretty clean, the information is completely denormalized as you could see. For that matter a collection of methods (functions) are available in order to generate the tables you might need:

- `row_filter(df, cat_var, cat_values)` => Filter rows by any value or group of values in a categorical variable.

- `nrow_filter(df, cat_var, cat_values)` => The same but backwards. 

- `groupby_sum(df, group_vars, agg_var='Total', sort_var='Total')` => Add deaths by a certain variable.

- `pivot_table(df, col, x_axis, value='Total')`=> Make some pivot tables, you might need them...

__NOTE:__ be aware that the filtering methods can perform a filter at a time. Feel free to perform the filter you need in any way you want or feel confortable with.

In [9]:
# Example 1

dataset = mod.row_filter(deaths, 'Sexo', ['Total'])
dataset = mod.row_filter(dataset, 'Edad', ['Todas las edades'])
dataset.head()


,Causa de muerte,Sexo,Edad,Periodo,Total,cause_code,cause_group,cause_name
0,001-102 I-XXII.Todas las causas,Total,Todas las edades,2018,427721,001-102,Multiple causes,I-XXII.Todas las causas
1,001-102 I-XXII.Todas las causas,Total,Todas las edades,2017,424523,001-102,Multiple causes,I-XXII.Todas las causas
2,001-102 I-XXII.Todas las causas,Total,Todas las edades,2015,422568,001-102,Multiple causes,I-XXII.Todas las causas
3,001-102 I-XXII.Todas las causas,Total,Todas las edades,2016,410611,001-102,Multiple causes,I-XXII.Todas las causas
4,001-102 I-XXII.Todas las causas,Total,Todas las edades,2012,402950,001-102,Multiple causes,I-XXII.Todas las causas


In [10]:
# Example 2
group = ['cause_code','Periodo']
dataset = mod.groupby_sum(deaths, group)
dataset.head()


,cause_code,Periodo,Total
0,001-102,2018,1710884
1,001-102,2017,1698092
2,001-102,2015,1690272
3,001-102,2016,1642444
4,001-102,2012,1611800


In [11]:
# Example 3

dataset = mod.pivot_table(dataset, 'cause_code', 'Periodo')
dataset.head()


cause_code,Periodo,001,001-008,001-102,002,003,004,005,006,007,...,093,094,095,096,097,098,099,100,101,102
0,1980,1620,15768,1157376,5904,2008,3448,436,0,0,...,4956,1432,184,692,16748,6608,1496,28,968,96
1,1981,1404,15124,1173544,6332,1656,3344,348,0,0,...,4700,1200,156,1396,17472,6872,1284,336,908,208
2,1982,1308,13488,1146620,5352,1240,3104,316,0,0,...,4864,956,200,1000,18616,7404,1228,440,1132,52
3,1983,1212,13100,1210276,5152,1072,3152,336,0,0,...,4788,1464,148,884,18392,8724,1560,1276,1500,56
4,1984,1228,12928,1197636,4564,964,3704,424,0,0,...,4716,1244,164,1020,14696,9972,1812,1144,1636,76


## ...and finally, show me some insights with Plotly!!!

In [12]:
# Cufflinks histogram

year = 1982
age = 'Todas las edades'
sex = 'Total'
cause = 'Single cause'
mask = (deaths['Periodo'] == year) & (deaths['Edad'] == age) & (deaths['Sexo'] == sex) & (deaths['cause_group'] == cause)
               
deaths_year_age_sex = deaths[mask]
deaths_sorted = deaths_year_age_sex.sort_values('Total', ascending=False).head(10)
deaths_sorted

deaths_sorted.iplot(kind='barh',
                    x='cause_name',
                    y='Total',
                    title='Mostly causes',
                    yTitle='Cause',
                    xTitle='Number')


In [13]:
year = 1982
sex = 'Total'
age = 'Todas las edades'
cause = 'Single cause'
mask = (deaths['Periodo'] == year) & (deaths['Sexo'] == sex) & (deaths['cause_group'] == cause)  & (deaths['Edad'] != age)
deaths_year_sex = deaths[mask]
deaths_year_sex
deaths_max = deaths_year_sex.loc[deaths_year_sex.groupby(['Edad'])['Total'].idxmax().reset_index(drop=True)]
#len(deaths_max)
deaths_max.sort_values('Edad')


graph = px.bar(deaths_max, 
               x = "cause_name", 
               y = "Total", 
               color = "Edad", 
               title = "Mostly causes 1982",
               width=900, height=1500)
graph.show()


In [ ]:
# Cufflinks line plot
'''
dataset_line.iplot(kind='line',
                   x='VARIABLE',
                   xTitle='AXIS TITLE',
                   yTitle='AXIS TITLE',
                   title='VIZ TITLE')
'''

In [ ]:
# Cufflinks scatter plot
'''
dataset_scatter.iplot(x='VARIABLE', 
                      y='VARIABLE', 
                      categories='VARIABLE',
                      xTitle='AXIS TITLE', 
                      yTitle='AXIS TITLE',
                      title='VIZ TITLE')
'''